In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

data_jeremy = pd.read_csv('scraper/data.csv')

links = list(set(list(data_jeremy['link'])))
users = list(set(list(data_jeremy['Facebook user id'])))

print "Links"
print links[:5]
print "Users"
print users[:5]

matrix = pd.DataFrame(np.zeros((len(links), len(users))))
matrix.index = links
matrix.columns = users

print "matrix_size:",
print matrix.shape

for i in data_jeremy.iterrows():
    matrix.loc[i[1][0], i[1][2]] = 1

start = time.time()
for count, i in enumerate(matrix):
    if count % (len(matrix)/10) == 0:
        print 'Count = %r/%r' %(count,len(matrix))
    if sum(matrix[i]) == 1:
        matrix.drop(i, 1, inplace=True)
print 'Time for drop = %r' %(time.time() - start)

def PCA(matrix):
    A = matrix
    M = (A-np.mean(A.T,axis=1)).T # subtract the mean (along columns)
    [latent,coeff] = np.linalg.eig(np.cov(M)) # attention:not always sorted
    score = np.dot(coeff.T,M) # projection of the data in the new space
    return coeff,score,latent

matrix = matrix.T
coef, score, eigenvalues = PCA(matrix)

principal_components = coef[:,:2]
print "Principal Components"
print principal_components
print "Eigenvalues"
print eigenvalues
compressed = np.dot(matrix, principal_components)


Links
['http://news-beta.nationalgeographic.com/2015/01/150128-big-bang-universe-supernova-astrophysics-health-space-ngbooktalk/?utm_source=Facebook&utm_medium=Social&utm_content=link_fb20150128news-spacebooktalk&utm_campaign=Content&sf7156883=1', 'https://www.youtube.com/watch?v=P7dEJuBKG28', 'http://inhabitat.com/vancouver-pledges-to-switch-to-100-percent-renewable-energy/', 'http://www.nytimes.com/2015/09/29/science/space/mars-life-liquid-water.html', 'http://montrealgazette.com/business/local-business/montreal-based-startup-breather-raises-20m-in-funding']
Users
[10101283842875395, 10102003312058373, 10105082978836486, 10156265407635463, 10150568216829960]
matrix_size: (3032, 4741)
Count = 0/3032
Count = 303/3032
Count = 606/3032
Count = 909/3032
Count = 1212/3032
Count = 1515/3032
Count = 1818/3032
Count = 2121/3032
Count = 2424/3032
Count = 2727/3032
Count = 3030/3032
Count = 3333/3032
Count = 3636/3032
Count = 3939/3032
Count = 4242/3032
Count = 4545/3032
Time for drop = 116.757

In [84]:
principal_components = coef[:,:10]
compressed = np.array(np.dot(matrix, principal_components), dtype=float)

/Users/gnil/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:2: ComplexWarning: Casting complex values to real discards the imaginary part
  from IPython.kernel.zmq import kernelapp as app


In [85]:
from scipy.cluster.vq import vq, kmeans, whiten

compressed_centroids = kmeans(compressed, 15)
centroids = []
for centroid in compressed_centroids[0]:
    centroids.append(np.dot(principal_components, centroid))

In [106]:
import math

def l2_norm(vector1, vector2):
    return math.sqrt(sum(v**2 for v in (vector1 - vector2)))

def recommend_link(user):
    max_distance = 0
    recommendation = None
    for centroid in centroids:
        if np.linalg.norm(user - centroid) > max_distance:
            max_distance = np.linalg.norm(user - centroid)
            recommendation = links[np.argmax(centroid)]
    return recommendation



In [107]:
users[0]

10101283842875395

In [111]:
for i in range(1000):
    print recommend_link(users[i])

http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://islands.im
http://isl

In [95]:
for centroid in centroids:
    print np.linalg.norm(centroid)
    print links[np.argmax(centroid)]

0.70787829801
http://islands.im
0.0194591271218
http://www.dailymail.co.uk/news/article-2735424/Roll-times-U-S-company-takes-cardboard-OUT-toilet-paper-time-century-cut-waste.html
0.211234378336
http://www.dailymail.co.uk/news/article-2735424/Roll-times-U-S-company-takes-cardboard-OUT-toilet-paper-time-century-cut-waste.html
2.70454700233
http://www.hks.harvard.edu/degrees/teaching-courses/course-listing/api-302
0.701494940477
http://www.extremetech.com/extreme/123485-mit-discovers-the-location-of-memories-individual-neurons
5.38146660197
http://www.boredpanda.com/world-leaders-pooping-the-daily-duty-cristina-guggeri/
0.169011889844
http://www.hks.harvard.edu/degrees/teaching-courses/course-listing/api-302
0.0899638848903
http://classics.esquire.com
1.31371319612
http://islands.im
1.40273950007
http://www.forbes.com/30under30/#/
10.8707049782
http://www.bostonglobe.com/lifestyle/2015/05/10/the-benefits-slow-parenting/2LImOAIyqElORCStgOADSI/story.html?event=event25
17.4102853715
http://